In [1]:
import os

In [2]:
%pwd

'C:\\Users\\shrey\\OneDrive\\Desktop\\MLOPS PORJECT\\Wine-Quality-Prediction-MLOps-Project\\research'

In [3]:
os.chdir("../")
%pwd

'C:\\Users\\shrey\\OneDrive\\Desktop\\MLOPS PORJECT\\Wine-Quality-Prediction-MLOps-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class dataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [21]:
from src.WineQuality.constants import *
from src.WineQuality.utils.common import read_yaml, create_directories
from src.WineQuality.entity.config_entity import DataIngestionConfig


class ConfigurationManager:

    def __init__(self, config_filePath = CONFIG_FILE_PATH, params_filePath = PARAMS_FILE_PATH, schema_filePath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filePath)
        self.params = read_yaml(params_filePath)
        self.schema = read_yaml(schema_filePath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [22]:
import os
from urllib.request import urlretrieve
from src.WineQuality import logger
from src.WineQuality.entity.config_entity import DataIngestionConfig
import zipfile


class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)

            filename, headers = urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

            
    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-07 01:00:16,721: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-07 01:00:16,723: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-07 01:00:16,728: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-07 01:00:16,731: INFO: common: created directory at: artifacts]
[2025-08-07 01:00:16,732: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-07 01:00:17,880: INFO: 3175180489: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AF4D:23737A:192EA9:3A71EB:6893AD4B
Accept-Ranges: bytes
Date: Wed, 